In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import agentpy as ap
import seaborn as sns
import IPython


In [2]:
class CombineHarvester(ap.Agent):
    def setup(self):
        # Initialize harvester with wheat capacity, current load, fuel capacity, current fuel level, starting position, and active status.
        self.capacity = 5  # Maximum amount of wheat the harvester can carry.
        self.load = 0  # Current amount of wheat in the harvester.
        self.gas_capacity = 20  # Total fuel capacity of the harvester.
        self.gas_level = self.gas_capacity  # Current fuel level, starts full.
        self.position = (0, 0)  # Starting position of the harvester on the field.
        self.is_active = True  # Indicates whether the harvester is operational.

    def move(self):
        # Controls the movement of the harvester.
        # Deactivate the harvester if it is inactive, its load is full, or it runs out of fuel.
        if not self.is_active or self.load >= self.capacity or self.gas_level <= 0:
            self.is_active = False
            return

        x, y = self.position
        # Movement logic: The harvester moves primarily vertically (up and down) and shifts right at the end of each column.
        if y < self.model.field_size - 1 and not self.model.moving_down:
            self.position = (x, y + 1)  # Move down vertically.
        elif y > 0 and self.model.moving_down:
            self.position = (x, y - 1)  # Move up vertically.
        else:
            # Shift right at the end of a column and reverse vertical movement direction.
            if x < self.model.field_size - 1:
                self.position = (x + 1, y)
                self.model.moving_down = not self.model.moving_down
            else:
                self.gas_level = 0  # Set fuel to zero if it reaches the end of the field.

        # Fuel consumption logic.
        if self.gas_level > 0:
            self.gas_level -= 1  # Reduce fuel by one unit per move.
        else:
            self.is_active = False  # Deactivate if out of fuel.

    def harvest(self):
        # Harvest wheat at the current position if the harvester has capacity and fuel.
        if self.load < self.capacity and self.gas_level > 0:
            x, y = self.position
            # Check for wheat presence at the current position and harvest it.
            if self.model.field[y][x]:  # Note: Accessing field using [y][x] due to orientation.
                self.model.field[y][x] = False  # Mark the wheat as harvested.
                self.load += 1  # Increase the wheat load.
        # Check conditions to deactivate the harvester.
        if self.load >= self.capacity or self.gas_level <= 0:
            self.is_active = False  # Deactivate if full or out of fuel.
        elif self.gas_level > 0:
            self.is_active = True  # Reactivate harvester if it still has fuel.


In [3]:
class Truck(ap.Agent):
    def setup(self):
        # Initialize the truck with infinite fuel capacity, its current position, and its active status.
        self.gas_capacity = float("inf")  # The truck has an unlimited fuel capacity.
        self.gas_level = self.gas_capacity  # Current fuel level, starts at maximum.
        self.position = (0, 0)  # Starting position next to the harvester.
        self.is_active = False  # Indicates if the truck is currently operational.
        self.speed = 1  # Speed at which the truck can move.

    def move_towards_position(self, target_position):
        # Method to move the truck towards a specified position.
        if self.position != target_position:
            # Calculate the direction to move based on the target position.
            direction = (np.sign(target_position[0] - self.position[0]),
                         np.sign(target_position[1] - self.position[1]))
            # Update the truck's position in the direction of the target.
            self.position = (self.position[0] + direction[0] * self.speed,
                             self.position[1] + direction[1] * self.speed)

    def move_to_harvester(self, harvester):
        # Method to move the truck towards the harvester if active and has fuel.
        if self.is_active and self.gas_level > 0:
            self.move_towards_position(harvester.position)
            self.gas_level -= 1  # Decrease fuel level, even though it's infinite.

    def check_harvester_status(self, harvester):
        # Check if the harvester needs support based on its fuel and load levels.
        if harvester.gas_level <= harvester.gas_capacity * 0.25 or harvester.load >= harvester.capacity * 0.75:
            self.is_active = True  # Activate the truck if the harvester needs support.
        else:
            self.is_active = False  # Deactivate the truck if the harvester doesn't need support.

    def collect(self, harvester):
        # Method for the truck to collect wheat or refuel the harvester.
        if self.position == harvester.position:
            # Check if the truck is at the harvester's position and if the harvester needs unloading or refueling.
            if harvester.load >= harvester.capacity * 0.75 or harvester.gas_level <= 0:
                harvester.load = 0  # Empty the harvester's load.
                harvester.gas_level = harvester.gas_capacity  # Refuel the harvester.
                harvester.is_active = True  # Reactivate the harvester after servicing.
                self.is_active = False  # Reset the truck's status.


In [4]:
class HarvestModel(ap.Model):
    def setup(self):
        # Initialize the simulation environment.
        self.field_size = 20  # Size of the field (20x20 grid).
        self.max_steps = 100  # Limit the simulation to a maximum of 100 steps.
        self.current_step = 0  # Counter to track the number of steps taken.

        # Create a field represented as a 2D grid where each cell can either have wheat (True) or not (False).
        self.field = np.full((self.field_size, self.field_size), True)  # Initially, all cells have wheat.

        # Randomly remove wheat from a certain percentage of the field.
        wheat_coverage = 0.9  # 90% of the field will initially have wheat.
        num_no_wheat = int((1 - wheat_coverage) * self.field_size * self.field_size)  # Number of cells without wheat.
        no_wheat_indices = self.random.sample(list(np.ndindex(self.field.shape)), num_no_wheat)  # Randomly select cells.
        for idx in no_wheat_indices:
            self.field[idx] = False  # Remove wheat from these selected cells.

        # Initialize the combine harvester and the truck agents.
        self.combine = CombineHarvester(self)
        self.truck = Truck(self)
        self.moving_down = False  # A flag to control the combine's vertical movement.

    def step(self):
        # This method is called at each step of the simulation.
        self.current_step += 1  # Increment the step counter.

        # Check the status of the harvester and perform actions accordingly.
        self.truck.check_harvester_status(self.combine)  # Truck checks if the harvester needs assistance.

        # If the harvester has fuel, it moves and harvests wheat.
        if self.combine.gas_level > 0:
            self.combine.move()
            self.combine.harvest()

        # The truck moves towards the harvester and collects wheat or refuels it.
        self.truck.move_to_harvester(self.combine)
        self.truck.collect(self.combine)

        # Stop conditions for the simulation.
        # The simulation stops if all wheat is harvested or if the combine harvester runs out of fuel.
        if not np.any(self.field) or self.combine.gas_level <= 0:
            self.stop()

        # Stop the simulation if it reaches the maximum number of steps.
        if self.current_step >= self.max_steps:
            self.stop()

    def end(self):
        # This method is called at the end of the simulation.
        self.report('Harvesting completed')  # Report the completion of the harvesting process.


In [5]:
def animation_plot(model, ax):
    # Function to update the plot for each frame of the animation.
    field_grid = np.array(model.field)  # Convert the field data to a NumPy array for visualization.
    ax.clear()  # Clear the current axes to redraw the updated frame.

    # Display the field as an image where the presence of wheat is indicated by color.
    ax.imshow(field_grid, cmap='YlGn_r', interpolation='none', alpha=0.8)

    # Plot the positions of the combine harvester and the truck.
    ax.scatter(*model.combine.position, color='#22dd33', label='Combine Harvester', s=100)
    ax.scatter(*model.truck.position, color='blue', label='Truck', s=100)

    # Set the title with current status information.
    ax.set_title(f"Harvester gasoline: {model.combine.gas_level}\nHarvester wheat: {model.combine.load}\nTime-step: {model.t}")

    # Remove the axis numbers (ticks).
    ax.set_xticks([])  # Remove x-axis ticks.
    ax.set_yticks([])  # Remove y-axis ticks.

    ax.legend()  # Display the legend.


parameters = {'field_size': 20}
fig, ax = plt.subplots()
model = HarvestModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot, interval=100)  # Limit the number of frames
IPython.display.HTML(animation.to_jshtml())